# Proyecto 1 NLP – Clasificación supervisada

**Universidad del Valle de Guatemala**  
**Facultad de Ingeniería**  
**Departamento de Ciencias de la Computación**  
**Procesamiento de Lenguaje Natural**   

## Integrantes: 
- Pablo Orellana
- Diego Leiva
- Renatto Guzmán

## Librerías

In [ ]:
import pandas as pd
import re, unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SpanishStemmer
import nltk
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import hstack, coo_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

## 1. Preprocesamiento del Corpus

### Cargar Corpus

Corpus obtenido de Kaggle en: [IMDB Dataset of 50K Movie Reviews](https://www.kaggle.com/datasets/luisdiegofv97/imdb-dataset-of-50k-movie-reviews-spanish)

In [ ]:
df = pd.read_csv('Data/IMDB Dataset SPANISH.csv')
print(f"=== Dataset ===")
print(f"Documentos: {df.shape[0]}")
print(f"Características: {df.shape[1]}\n")
print("=== Información del Dataset ===")
df.info()
print("\n=== Primeras 5 filas del Dataset ===")
df.head()

### Procesamiento de DataFrame

In [ ]:
# Eliminar columnas en ingles o innecesarias
df = df.drop(columns=['review_en', 'Unnamed: 0', 'sentiment'])

# Renombrar columnas
df = df.rename(columns={'review_es': 'review'})

# Convertir variable objetivo a binaria
df['sentimiento'] = df['sentimiento'].map({'positivo': 1, 'negativo': 0})

# Eliminar duplicados
df = df.drop_duplicates()

# Eliminar filas con valores nulos
df = df.dropna()

print(f"=== Dataset Procesado ===")
print(f"Documentos: {df.shape[0]}")
print(f"Características: {df.shape[1]}\n")
print("\n=== Primeras 5 filas del Dataset ===")
df.head()

### Normalización de Texto

In [ ]:
def limpiar_texto(texto: str) -> str:
    """
    Función para limpiar y normalizar texto en español.
    Parámetros:
        texto (str): Texto a limpiar.
    Retorna:
        str: Texto limpio y normalizado.
    """
    # 1) a minúsculas
    s = str(texto).lower().strip()
    # 2) quitar tildes/diacríticos
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    # 3) remover URLs, emails, @usuarios, #hashtags
    s = re.sub(r'https?://\S+|www\.\S+', ' ', s)
    s = re.sub(r'\S+@\S+\.\S+', ' ', s)
    s = re.sub(r'[@#]\w+', ' ', s)
    # 4) quitar puntuación y símbolos
    s = re.sub(r'[^\w\s]', ' ', s, flags=re.UNICODE)  # elimina ¡¿, comas, puntos, comillas, etc.
    s = re.sub(r'_', ' ', s)
    # 5) espacios múltiples a uno
    s = re.sub(r'\s+', ' ', s)
    # 6) eliminar numeros
    s = re.sub(r'\d+', ' ', s)
    return s.strip()

In [ ]:
# Crear columna normalizada
df["review_norm"] = df["review"].apply(limpiar_texto)

# Mostrar ejemplos de limpieza
print("Ejemplo (antes y después):\n")
for i in range(5):
    print(f"- Original: {df['review'].iloc[i][:50]}...")
    print(f"- Limpia  : {df['review_norm'].iloc[i][:50]}...\n")

# Verificar si hay vacios después de la limpieza
vacios = df['review_norm'].str.strip().eq('')
print(f"Número de filas con texto vacío después de la limpieza: {vacios.sum()}")
# Eliminar filas con texto vacío
df = df[~vacios]
print(f"Filas restantes después de eliminar textos vacíos: {df.shape[0]}")

### Stopwords
Se amplian las stopwords de NTLK con el conjunto de stopwords de [alir3z4 Stop Words](https://alir3z4.github.io/stop-words/)

In [ ]:
def normalize(s: str) -> str:
    """
    Normaliza una cadena de texto, convirtiéndola a minúsculas, eliminando tildes y espacios en blanco.
    Parámetros:
        s (str): Cadena de texto a normalizar.
    Retorna:
        str: Cadena de texto normalizada.
    """
    s = s.lower().strip()
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def norm_set(words):
    """
    Normaliza un conjunto de palabras.
    Parámetros:
        words (iterable): Conjunto de palabras a normalizar.
    Retorna:
        set: Conjunto de palabras normalizadas.
    """
    return {normalize(w) for w in words if w.strip()}

In [ ]:
# Descargar stopwords de NLTK si no se han descargado
nltk.download('stopwords', quiet=True)

# 1) base y extras
sw_es = set(stopwords.words('spanish'))
sw_en = set(stopwords.words('english'))

# Agregar stopwords comunes en español e inglés extra
with open("Data/spanish_stopwords.txt", encoding="utf-8") as f:
    es_extra = {ln.strip() for ln in f if ln.strip()}
with open("Data/english_stopwords.txt", encoding="utf-8") as f:
    en_extra = {ln.strip() for ln in f if ln.strip()}

# 2) protegidas
negaciones = {
    "no","nunca","jamas","ni","sin","tampoco","ningun",
    "ninguna","ninguno","ningunas","ningunos","nadie","nada"
    }
contrastivos = {
    "pero","aunque","sin","embargo","sinembargo","no",
    "obstante","sino","excepto","salvo","aunquesi"
    }
intensif = {"muy","tan","tanto","tantos","tantas",
            "demasiado","demasiada","demasiados",
            "demasiadas","super","re","hiper","bastante",
            "apenas","poco","poca","pocos","pocas","casi",
            "algo","sumamente"
            }

protegidas = norm_set(negaciones | contrastivos | intensif)

# 3) normalizar todo
sw_total_norm = norm_set(sw_es|es_extra|sw_en|en_extra) - protegidas
print("=== Resumen de stopwords ===")
print(f"Stopwords español base: {len(sw_es)} | extra: {len(es_extra)}")
print(f"Stopwords inglés base: {len(sw_en)} | extra: {len(en_extra)}")
print(f"Stopwords finales: {len(sw_total_norm)}")

# 4) regex con bordes de palabra
pattern = re.compile(r'\b(?:' + '|'.join(sorted(map(re.escape, sw_total_norm), key=len, reverse=True)) + r')\b')

# 5) aplicar a la columna ya normalizada
df["review_sin_sw"] = (
    df["review_norm"]
      .str.replace(pattern, " ", regex=True)
      .str.replace(r"\s+", " ", regex=True)
      .str.strip()
)

# Chequeo rápido
print("\nEjemplos (antes y sin stopwords):\n")
for i in range(5):
    print(f"- Reseña        : {df['review_norm'].iloc[i][:50]}...")
    print(f"- Sin Stopwords : {df['review_sin_sw'].iloc[i][:50]}...\n")

# Verificar si hay vacios después de remover stopwords
vacios_sw = df['review_sin_sw'].str.strip().eq('')
print(f"Número de filas con texto vacío después de remover stopwords: {vacios_sw.sum()}")
# Eliminar filas con texto vacío
df = df[~vacios_sw]
print(f"Filas restantes después de eliminar textos vacíos: {df.shape[0]}")

### Tokenización y Stemming

In [ ]:
# Descargar recursos de NLTK si no se han descargado
nltk.download('punkt', quiet=True)

# Inicializar el stemmer en español
stemmer = SpanishStemmer()

# Aplicar stemming a la columna sin stopwords
# tokens crudos (sin stopwords)
df["tokens"] = df["review_sin_sw"].apply(lambda x: word_tokenize(x, language="spanish"))

# tokens stem (lista)
df["tokens_stem"] = df["tokens"].apply(lambda xs: [stemmer.stem(t) for t in xs])

df.head()

### Levenshtein
No se aplica a todo el dataset debido a que:
- El corpus tiene ~50 000 reseñas → millones de palabras únicas.
- Comparar cada palabra con todas las demás sería $(O(n^2))$, ineficiente y sin valor práctico.

In [ ]:
def levenshtein(a: str, b: str) -> int:
    """
    Distancia de edición mínima entre a y b.
    
    Parámetros:
        a (str): Primera cadena.
        b (str): Segunda cadena.
    Retorna:
        int: Distancia de edición mínima.
        """
    if a == b:
        return 0
    if not a:
        return len(b)
    if not b:
        return len(a)
    dp = np.zeros((len(a)+1, len(b)+1), dtype=int)
    dp[0, :] = np.arange(len(b)+1)
    dp[:, 0] = np.arange(len(a)+1)
    for i in range(1, len(a)+1):
        for j in range(1, len(b)+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i, j] = min(dp[i-1, j]+1, dp[i, j-1]+1, dp[i-1, j-1]+cost)
    return dp[len(a), len(b)]

In [ ]:
# Construcción del vocabulario
# Tomamos los tokens sin stem para detectar errores reales
vocab_counts = Counter(t for toks in df["tokens"] for t in toks)
print(f"Vocabulario total: {len(vocab_counts)} palabras")

# Palabras frecuentes (consideradas válidas)
freq_words = {w for w, c in vocab_counts.items() if c >= 5}

# Ejemplo de corrección
palabra = "exelente"
distancias = {w: levenshtein(palabra, w) for w in list(freq_words)[:2000]}  # comparar con un subconjunto
sugerencia = min(distancias, key=distancias.get)
print(f"Palabra: {palabra} → Sugerencia más cercana: {sugerencia}")

# Opcional: aplicar a todo el vocabulario raro (<3 apariciones)
rare = {w for w, c in vocab_counts.items() if c < 3}
correcciones = {}
for w in list(rare)[:20]:
    candidatos = {v: levenshtein(w, v) for v in freq_words if abs(len(v)-len(w)) <= 2}
    if candidatos:
        best = min(candidatos, key=candidatos.get)
        if candidatos[best] <= 1:
            correcciones[w] = best

print("\nEjemplos de correcciones detectadas:")
for k, v in list(correcciones.items())[:10]:
    print(f"{k} → {v}")

## 2. Representación del Texto

### Bolsa de Palabras (BoW) y TF-IDF

In [ ]:
# Crear reseñas con "stems" para vectorizadores basados en conteo
df["text_stem"] = df["tokens_stem"].apply(lambda xs: " ".join(xs))

# Aplicar BoW y TF-IDF
bow_vec = CountVectorizer(min_df=5, max_df=0.5, ngram_range=(1,2))   # unigrams+bigramas
tfidf_vec = TfidfVectorizer(min_df=5, max_df=0.5, ngram_range=(1,2), sublinear_tf=True)

# Transformar
X_bow   = bow_vec.fit_transform(df["text_stem"])
X_tfidf = tfidf_vec.fit_transform(df["text_stem"])

print("=== BoW ===")
print(f"Documentos: {X_bow.shape[0]}")
print(f"Términos distintos: {X_bow.shape[1]}")
print(f"Densidad promedio: {X_bow.nnz / (X_bow.shape[0]*X_bow.shape[1]):.6f}")
print(f"Entradas no nulas (nnz): {X_bow.nnz:,}")
print(f"Promedio de términos no nulos por documento: {X_bow.nnz / X_bow.shape[0]:.1f}\n")

print("=== TF-IDF ===")
print(f"Documentos: {X_tfidf.shape[0]}")
print(f"Términos distintos: {X_tfidf.shape[1]}")
print(f"Densidad promedio: {X_tfidf.nnz / (X_tfidf.shape[0]*X_tfidf.shape[1]):.6f}")
print(f"Entradas no nulas (nnz): {X_tfidf.nnz:,}")
print(f"Promedio de términos no nulos por documento: {X_tfidf.nnz / X_tfidf.shape[0]:.1f}")

### Matriz de co-ocurrencia y aplicación de PPMI

In [ ]:
# Limitar a vocabulario top-V para controlar memoria
V = 10000  # Solo 10k palabras más frecuentes
# Obtener mas frecuentes
freq = Counter(t for toks in df["tokens"] for t in toks).most_common(V)
itos = [w for w,_ in freq]
stoi = {w:i for i,w in enumerate(itos)}

# Contar co-ocurrencias en ventana
window = 4     # contexto a cada lado
pair_counts = defaultdict(int)

# Iterar sobre tokens y contar pares
for toks in df["tokens"]:
    # obtener índices de vocabulario
    idxs = [stoi[t] for t in toks if t in stoi]
    # contar pares en ventana
    for i, wi in enumerate(idxs):
        j0 = max(0, i-window) # inicio de ventana
        j1 = min(len(idxs), i+window+1) # fin de ventana
        # contar pares
        for j in range(j0, j1):
            if j == i: 
                continue
            wj = idxs[j]
            if wi == wj: 
                continue
            # conteo simétrico
            a,b = (wi,wj) if wi < wj else (wj,wi)
            pair_counts[(a,b)] += 1 

# Construir matriz simétrica COO
rows, cols, data = [], [], [] # listas para COO

# llenar listas
for (i,j), c in pair_counts.items():
    rows += [i, j]
    cols += [j, i]
    data += [c, c]
# Crear matriz dispersa COO
C = coo_matrix((data, (rows, cols)), shape=(V, V), dtype=np.float64).tocsr()
total = C.sum() # suma total de co-ocurrencias

# Calcular PPMI
# PMI(i,j) = log2( P(i,j) / (P(i)P(j)) ), PPMI = max(PMI, 0)
row_sums = np.asarray(C.sum(axis=1)).ravel()
col_sums = row_sums  # simétrica
C_coo = C.tocoo()
p_ij = C_coo.data / total
p_i  = row_sums[C_coo.row] / total
p_j  = col_sums[C_coo.col] / total
pmi  = np.log2((p_ij / (p_i * p_j)) + 1e-12)
ppmi_data = np.maximum(pmi, 0.0)

# Crear matriz PPMI dispersa
PPMI = coo_matrix((ppmi_data, (C_coo.row, C_coo.col)), shape=C.shape).tocsr()
print("=== MATRIZ DE CO-OCURRENCIA ===")
print(f"Dimensión: {C.shape}")
print(f"Entradas no nulas (nnz): {C.nnz:,}")
print(f"Densidad: {C.nnz / (C.shape[0]*C.shape[1]):.8f}")
print(f"Total de co-ocurrencias contadas: {int(total):,}")
print(f"Media de conteo por par no nulo: {C.data.mean():.2f}")

print("\n=== MATRIZ PPMI ===")
print(f"Dimensión: {PPMI.shape}")
print(f"Entradas no nulas (nnz): {PPMI.nnz:,}")
print(f"Densidad: {PPMI.nnz / (PPMI.shape[0]*PPMI.shape[1]):.8f}")
print(f"Valor medio PPMI (>0): {PPMI.data.mean():.4f}")
print(f"Valor máximo PPMI: {PPMI.data.max():.4f}")

### Implementación de embeddings con Word2Vec

In [ ]:
# Preparar datos para Word2Vec/FastText
sentences = df["tokens"].tolist()

# Word2Vec con skip-gram
w2v = Word2Vec(
    sentences=sentences, 
    vector_size=100, 
    window=5, 
    min_count=5, 
    workers=4, 
    sg=1, 
    negative=10, 
    epochs=5
    )

# Vectores de documento por promedio de palabras presentes
def doc_vector(tokens, kv):
    vecs = [kv[w] for w in tokens if w in kv]
    if not vecs:
        return np.zeros(kv.vector_size, dtype=np.float32)
    return np.mean(vecs, axis=0)

doc_vecs_w2v = np.vstack([doc_vector(toks, w2v.wv) for toks in sentences])
palabras_ejemplo = ["bueno", "malo", "excelente", "terrible", "divertido", "aburrido"]

# Mostrar dimendiones y ejemplos
print("=== Word2Vec ===")
print(f"Tamaño del vocabulario w2v: {len(w2v.wv)}")
print(f"Dimensiones de los vectores w2v: {w2v.wv.vectors.shape[1]}")
print("Ejemplos: ")
for w in palabras_ejemplo:
    if w in w2v.wv:
        print(f"Similares a '{w}':", [p for p,_ in w2v.wv.most_similar(w)[:5]])
    else:
        print(f"{w}: No en vocabulario")

### Comparación mediante PCA o t-SNE

In [ ]:
def plot_pca(X: np.ndarray, words: list, title: str) -> None:
    """
    Visualiza vectores en 2D usando PCA.
    Parámetros:
        X: Matriz de vectores (n_samples, n_features)
        words: Lista de palabras correspondientes a los vectores
        title: Título del gráfico
    Retorna: 
        None
    """
    pca = PCA(n_components=2, random_state=0)
    X2 = pca.fit_transform(X)
    plt.figure(figsize=(8,6))
    plt.scatter(X2[:,0], X2[:,1], s=8)
    for i,w in enumerate(words[:60]):  # anota pocas
        plt.annotate(w, (X2[i,0], X2[i,1]), fontsize=8)
    plt.title(title)
    plt.show()

def plot_tsne(X: np.ndarray, words: list, title: str, n: int = 200) -> None:
    """
    Visualiza vectores en 2D usando t-SNE.
    Parámetros:
        X: Matriz de vectores (n_samples, n_features)
        words: Lista de palabras correspondientes a los vectores
        title: Título del gráfico
        n: Número de muestras a visualizar
    Retorna: 
        None
    """
    Xs = X[:n]; ws = words[:n]
    tsne = TSNE(n_components=2, init="random", perplexity=30,
                learning_rate="auto", max_iter=1000, random_state=0)
    X2 = tsne.fit_transform(Xs)
    plt.figure(figsize=(8,6))
    plt.scatter(X2[:,0], X2[:,1], s=8)
    for i,w in enumerate(ws[:50]):
        plt.annotate(w, (X2[i,0], X2[i,1]), fontsize=8)
    plt.title(title)
    plt.show()

In [ ]:
# === Selección de palabras comunes para comparar W2V vs FastText ===
K = 300  # muestra
cand = [w for w,_ in freq]  # freq ya calculado antes
common = [w for w in cand if (w in w2v.wv.key_to_index) and (w in ft.wv.key_to_index)]
words = common[:K]
print(f"Palabras comunes usadas: {len(words)}")

# === Matrices de vectores ===
X_w2v = np.vstack([w2v.wv[w] for w in words])

# === W2V: PCA y t-SNE ===
plot_pca(X_w2v, words, "W2V palabras – PCA (muestra común)")
plot_tsne(X_w2v, words, "W2V palabras – t-SNE (muestra común)")